<a href="https://colab.research.google.com/gist/orans24/799b27e17b22ae43b2aec3a6456d3cf4/datatest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# drafts

# the final code that need to run for prediction

In [ ]:
#run the prediction
!pip install yfinance
!pip install pandas_market_calendars
!pip install trading_calendars
!pip install pytz
import ipywidgets as widgets
from IPython.display import display, clear_output
import pytz
import trading_calendars as tc
import yfinance as yf
import datetime as dt
import numpy as np
from numpy import load
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import math
from mlxtend.preprocessing import minmax_scaling
from datetime import datetime
import pandas_market_calendars as mcal
with tf.device('/device:GPU:0'):
  clear_output()
  def prediction(siu):
    np.seterr(divide='ignore', invalid='ignore')
    stocksnames=siu  #['AAPL','MSFT','FB','GOOGL']
    predictionstocks={}
    trends={};
    k=-1
    for sn in stocksnames:
      k=k+1
      stockssortedin = {};
      stockssortedout = {};
      mam = {}
      inputnotorder = [];
      normalhelp = [];
      vol = [];
      stockname = sn
      stock = yf.Ticker(stockname)
      maxdate = dt.datetime.now()
      max1 = str(maxdate)
      max1 = max1.split(' ')
      max = max1[0]
      min = '2021-01-01'
      data = yf.download(stockname, start=min, end=max)
      inputnotorder = data.to_numpy()
      inputnotorder = inputnotorder
      snp = yf.download('^GSPC', start=min, end=max)  # 1927
      inputnotorder1 = snp.to_numpy()
      djons = yf.download('^DJI', start=min, end=max)  # 1992-01-02
      inputnotorder2 = djons.to_numpy()
      nasdaq = yf.download('^IXIC', start=min, end=max)  # 1971
      inputnotorder3 = nasdaq.to_numpy()
      russ = yf.download('^RUT', start=min, end=max)  # 1987
      inputnotorder4 = russ.to_numpy()
      inputarr = [inputnotorder1, inputnotorder2, inputnotorder3, inputnotorder4]
      r1 = len(data)
      normalhelp = inputnotorder[r1-25:r1, 0:4].astype(np.float32)
      mini = np.min(normalhelp)
      maxi = np.max(normalhelp)
      mam[0] = [maxi, mini]
      normalhelp = (normalhelp - mini) / (maxi - mini)
      normalhelp = np.nan_to_num(normalhelp, nan=0.5, posinf=1, neginf=0)
      vol = inputnotorder[r1-25:r1, 4].astype(np.float32)
      vmin = np.min(vol)
      vmax = np.max(vol)
      vol = (vol - vmin) / (vmax - vmin)
      vol = np.nan_to_num(vol, nan=0.5, posinf=1, neginf=0)
      normalhelp = np.insert(normalhelp, 4, vol, axis=1)
      for arr in inputarr:
        normalhelp1 = arr[r1-25:r1, 0:4].astype(np.float32)
        mini = np.min(normalhelp1)
        maxi = np.max(normalhelp1)
        normalhelp1 = (normalhelp1 - mini) / (maxi - mini)
        normalhelp1 = np.nan_to_num(normalhelp1, nan=0.5, posinf=1, neginf=0)
        vol = arr[r1-25:r1, 5].astype(np.float32)
        vmin = np.min(vol)
        vmax = np.max(vol)
        vol = (vol - vmin) / (vmax - vmin)
        vol = np.nan_to_num(vol, nan=0.5, posinf=1, neginf=0)
        normalhelp1 = np.insert(normalhelp1, 4, vol, axis=1)
        normalhelp = np.append(normalhelp,normalhelp1,axis=1)
      stockssortedin[0] = normalhelp[0:25, :]
      x = list(stockssortedin.values())
      mm = list(mam.values())
      z1 = np.expand_dims(x[0], axis=0)
      model = tf.keras.models.load_model('/content/drive/MyDrive/PYTHON APP/mymodels400epoch/'+sn+'model')
      p = model(z1)
      maxi, mini = mm[0]
      p = p.numpy()
      p = (p * (maxi - mini)) + mini
      l = z1[0, 24, 3]
      l = (l * (maxi - mini)) + mini
      l1 = z1[0, :, 3]
      l1 = (l1 * (maxi - mini)) + mini
      if l == 0:
        break
      trendprediction = ((p[0, 9, 0] - l) / (l)) * 100
      predictionstocks[stocksnames[k]] = np.concatenate((l1, p), axis=None)
      trends[stocksnames[k]] = trendprediction
    yiyi=pd.DataFrame(trends,index=[1],columns=stocksnames)
    today = dt.date.today()
    nys = tc.get_calendar("NASDAQ")
    pas=list(nys.sessions_window(pd.Timestamp(nys.previous_close(pd.Timestamp(today)).strftime('%Y-%m-%d'), tz=pytz.UTC),-25).strftime('%m-%d'))
    fut=list(nys.sessions_window(pd.Timestamp(nys.previous_close(pd.Timestamp(today)).strftime('%Y-%m-%d'), tz=pytz.UTC),9).strftime('%m-%d'))
    dates=pas+fut[1:10]
    yaya=pd.DataFrame(predictionstocks,index=dates,columns=stocksnames)
    axes=yaya.plot(figsize=(25,10),subplots=True,grid=True,sharex=False)
    z=0

    for s in stocksnames:
      maxsp=yaya.loc[fut,s].max()
      minsp=yaya.loc[fut,s].min()
      maxdat=datetime.strptime(yaya.loc[fut,s].idxmax(),'%m-%d').strftime('%m-%d')
      mindat=datetime.strptime(yaya.loc[fut,s].idxmin(),'%m-%d').strftime('%m-%d')
      ddt=[mindat,maxdat]
      ssp=[minsp,maxsp]
      yoyo=pd.DataFrame(index=dates,columns=[s])
      yoyo.loc[mindat,s]=minsp
      yoyo.loc[maxdat,s]=maxsp
      yoyo.plot(ax=axes[z],grid=True,style=['y^-'])
      z=z+1
    plt.setp(axes,xticks=np.arange(len(dates)),xticklabels=dates)
    yaya.to_csv(r'/content/drive/MyDrive/PYTHON APP/predictions/'+max+'.csv')
    yiyi.to_csv(r'/content/drive/MyDrive/PYTHON APP/predictions/trendof' + max + '.csv')
    fig=plt.gcf()
    fig.savefig('/content/drive/MyDrive/PYTHON APP/predictions/'+max+'.png')
    yiyi.plot(figsize=(10,5),kind='bar')
    fig=plt.gcf()
    fig.savefig('/content/drive/MyDrive/PYTHON APP/predictions/trendof'+max+'.png')
  print("choose the stocks you want to predict(you can more then one):")
  stlist=widgets.SelectMultiple(options=['AAPL', 'MSFT', 'FB','GOOGL'],description='',disabled=False)
  display(stlist)
  button = widgets.Button(description='press to predict',disabled=False,button_style='success',tooltip='Click me',icon='check') 
  display(button)
  def foo(a):
    v=list(stlist.value)
    prediction(v)
  button.on_click(foo)
  

  

# training and building model

In [ ]:
#building and training net
from keras.models import Sequential
import tensorflow as tf
import os
import numpy as np
from tensorflow import keras
from keras.layers import LSTM,Dense,Reshape
from numpy import load
from sklearn.model_selection import train_test_split
with tf.device('/device:GPU:0'):
  stocksnames=['AAPL','FB','MSFT','GOOGL']
  for sn in stocksnames:
    model3=Sequential() 
    checkpoint_path = "/content/drive/MyDrive/PYTHON APP/training/cp.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_weights_only=True,
                                                  verbose=1)
    os.listdir(checkpoint_dir)
    
    dict_data = load('/content/drive/MyDrive/PYTHON APP/stocksdata/'+sn+'.npz')
    dict_data.allow_pickle=True
    x1=dict_data['arr_0']
    y1=dict_data['arr_1']
    x2=x1.tolist()
    y2=y1.tolist()
    x = list(x2.values())
    y = list(y2.values())
    x=np.nan_to_num(x,nan=0.5,posinf=1,neginf=0)
    y=np.nan_to_num(y,nan=0.5,posinf=1,neginf=0)
    x_train4,x_test4,y_train4,y_test4 = train_test_split(x, y,
    test_size=0.3,
    random_state=42)
    model3.add(LSTM(512,input_shape=(25,25),return_sequences=True,activation='tanh',recurrent_activation ='sigmoid'
    ,dropout=0.2,recurrent_dropout=0,use_bias=True,unroll=False))
    model3.add(LSTM(256,input_shape=(25,25),return_sequences=True,activation='tanh',recurrent_activation ='sigmoid'
    ,dropout=0.2,recurrent_dropout=0,use_bias=True,unroll=False))
    model3.add(LSTM(128,input_shape=(25,25),return_sequences=False,activation='tanh',recurrent_activation ='sigmoid'
    ,dropout=0.2,recurrent_dropout=0,use_bias=True,unroll=False))
    model3.add(Dense(10,activation='sigmoid',kernel_initializer=tf.initializers.zeros()))
    model3.add(Reshape([10,1]))
    model3.compile(loss='mse',
                  optimizer='adam',
                  metrics=['MeanAbsoluteError'])
    model3.fit(x_train4,
    y_train4,
    epochs=300
    ,verbose=1,
    validation_data=(x_test4,y_test4),batch_size=32,validation_batch_size=32,callbacks=[cp_callback])
    score = model3.evaluate(x_test4,y_test4,
    batch_size=32)
    model3.save('/content/drive/MyDrive/PYTHON APP/mymodels400epoch/'+sn+'model') 

# testing models trends accuracy and subplot presentation

In [ ]:
#subplot presentation
import matplotlib.pyplot as plt
fig,(axs1,axs2,axs3) = plt.subplots(3)
fig.set_figwidth(20)
fig.set_figheight(10)
axs1.plot(stockr[0:2500],color = 'red',linestyle='dotted',marker=',', label = 'real price')
axs1.plot(stockp[0:2500], color = 'black', label = 'Predicted ')
axs2.plot(stockr[2500:5000],color = 'red',marker=',',linestyle='dotted', label = 'real price')
axs2.plot(stockp[2500:5000], color = 'black', label = 'Predicted ')
axs3.plot(stockr[5000:len(stockp)],color = 'red',marker=',',linestyle='dotted', label = 'real price')
axs3.plot(stockp[5000:len(stockp)], color = 'black', label = 'Predicted ')
axs1.set_title('first 2500 days')
axs2.set_title('2500-5000 days')
axs3.set_title('5000 till now')
axs1.label_outer()
axs2.label_outer()
axs3.label_outer()
axs1.set(xlabel='days', ylabel='stock price')
axs2.set(xlabel='days', ylabel='stock price')
axs3.set(xlabel='days', ylabel='stock price')
axs1.legend()
axs2.legend()
axs3.legend()
fig.show()

In [ ]:
#checking trend accuracy
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras
from numpy import load

model=tf.keras.models.load_model('/content/drive/MyDrive/PYTHON APP/mymodels400epoch/APPLmodel')
dict_data = load('/content/drive/MyDrive/PYTHON APP/stocksdata/APPL.npz')
dict_data.allow_pickle=True
x1=dict_data['arr_0']
y1=dict_data['arr_1']
mm1=dict_data['arr_2']
x2=x1.tolist()
y2=y1.tolist()
mm2=mm1.tolist()
x = list(x2.values())
y = list(y2.values())
mm= list(mm2.values())
count=0
pretrends={}
realtreands={}
k={}
prestock={}
restock={}
for j in range(0,len(x)):
  z1=np.expand_dims(x[j], axis=0)
  f1=model(z1)
  p1=f1[0]
  p=p1[9,0]
  r1=y[j]
  r=r1[9]
  maxi,mini=mm[j]
  p=p.numpy()
  p=(p*(maxi-mini))+mini
  r=(r*(maxi-mini))+mini
  prestock[j]=p
  restock[j]=r
  l=z1[0,24,3]
  l=(l*(maxi-mini))+mini
  if l==0:
    break
  trendprediction=((p-l)/(l))*100
  realprediction=((r-l)/(l))*100
  pretrends[j]=trendprediction
  realtreands[j]=realprediction
  if trendprediction>1000:
    k[j]=[p,r,l]
  if trendprediction>=0:
    uppred=1
  else:
      uppred=0
  if realprediction>=0:
    upreal=1
  else:
      upreal=0
  acc=uppred+upreal
  if acc==2 or acc==0:
    count=count+1
print('from ',len(x),' accuracy trend are ',count)
stockp=list(prestock.values())
stockr=list(restock.values())
pre=list(pretrends.values())
real=list(realtreands.values())
f = plt.figure()
f.set_figwidth(200)
f.set_figheight(10)
plt.plot(stockr, 'red',linestyle='dotted',marker=',', label = 'real price')
plt.plot(stockp, 'black', label = 'Predicted ')
plt.title('real')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()
k

  




# create data for training models

In [ ]:
#building traning data for stock
# building traning data for stock


import yfinance as yf
import numpy as np
from numpy import load
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import math
from mlxtend.preprocessing import minmax_scaling
with tf.device('/device:GPU:0'):
    stockssortedin = {};
    stockssortedout = {};
    mam = {}
    inputnotorder = [];
    normalhelp = [];
    vol = [];
    np.seterr(divide='ignore', invalid='ignore')
    stocksnames = ['AAPL']
    for sn in stocksnames:
        stockssortedin = {};
        stockssortedout = {};
        mam = {}
        inputnotorder = [];
        normalhelp = [];
        vol = [];
        stockname = sn
        stock = yf.Ticker(stockname)
        data = stock.history(period="max")
        mindate = data.index.min()
        maxdate = data.index.max()
        min1 = str(mindate)
        min1 = min1.split(' ')
        min = min1[0]  # '1992-01-02'
        max1 = str(maxdate)
        max1 = max1.split(' ')
        max = max1[0]
        if min < '1992-01-02':
            min = '1992-01-02'
            data = yf.download(stockname, start=min, end=max)
        inputnotorder = data.to_numpy()
        inputnotorder = inputnotorder
        snp = yf.download('^GSPC', start=min, end=max)  # 1927
        inputnotorder1 = snp.to_numpy()
        djons = yf.download('^DJI', start=min, end=max)  # 1992-01-02
        inputnotorder2 = djons.to_numpy()
        nasdaq = yf.download('^IXIC', start=min, end=max)  # 1971
        inputnotorder3 = nasdaq.to_numpy()
        russ = yf.download('^RUT', start=min, end=max)  # 1987
        inputnotorder4 = russ.to_numpy()
        k = 0;
        h = 5
        inputarr = [inputnotorder1, inputnotorder2, inputnotorder3, inputnotorder4]

        r1 = len(data)
        for i in range(0, r1):
            if i >= r1 - 35:
                break
            normalhelp = inputnotorder[i:i + 35, 0:4].astype(np.float32)
            mini = np.min(normalhelp)
            maxi = np.max(normalhelp)
            mam[i] = [maxi, mini]
            normalhelp = (normalhelp - mini) / (maxi - mini)
            normalhelp = np.nan_to_num(normalhelp, nan=0.5, posinf=1, neginf=0)
            vol = inputnotorder[i:i + 35, 4].astype(np.float32)
            vmin = np.min(vol)
            vmax = np.max(vol)
            vol = (vol - vmin) / (vmax - vmin)
            vol = np.nan_to_num(vol, nan=0.5, posinf=1, neginf=0)
            normalhelp = np.insert(normalhelp, 4, vol, axis=1)
            for arr in inputarr:
                normalhelp1 = arr[i:i + 35, 0:4].astype(np.float32)
                mini = np.min(normalhelp1)
                maxi = np.max(normalhelp1)
                normalhelp1 = (normalhelp1 - mini) / (maxi - mini)
                normalhelp1 = np.nan_to_num(normalhelp1, nan=0.5, posinf=1, neginf=0)
                vol = arr[i:i + 35, 5].astype(np.float32)
                vmin = np.min(vol)
                vmax = np.max(vol)
                vol = (vol - vmin) / (vmax - vmin)
                vol = np.nan_to_num(vol, nan=0.5, posinf=1, neginf=0)
                normalhelp1 = np.insert(normalhelp1, 4, vol, axis=1)
                normalhelp = np.append(normalhelp,normalhelp1,axis=1)
                h=h+5
            stockssortedin[k] = normalhelp[0:25, :]
            stockssortedout[k] = normalhelp[25:35, 3]
            k = k + 1
        np.savez(stockname, stockssortedin, stockssortedout, mam)

print(np.max(stockssortedin.values()))


# mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
